In [1]:
# coding: utf-8
"""
train lightgbm model
"""
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd

import lightgbm as lgb

import gc
import pickle
from datetime import datetime
from scipy.sparse import hstack, csr_matrix
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import mean_squared_error

root = '/kaggle/competitions/avito-demand-prediction/'

def dump_matrix(matrix, name):
    pickle.dump(matrix, open(root + 'features/{}.pkl'.format(name), 'wb'), protocol=pickle.HIGHEST_PROTOCOL)
def load_matrix(name):
    return pickle.load(open(root + 'features/{}.pkl'.format(name), 'rb'))

In [2]:
tr_index = load_matrix('tr_index')
te_index = load_matrix('te_index')

In [3]:
daset_num_cols = load_matrix('daset_num_cols')

In [4]:
daset_num_cols.head()

,price,image_top_1,item_seq_number,n_missing_features,avg_days_up_user,avg_times_up_user,n_user_items,region_factor,parent_category_name_factor,user_type_factor,...,r_desc_wds,r_desc_dig,r_desc_cap,r_desc_spa,r_desc_pun,r_desc_row,r_desc_emo,r_titl_des,n_char_params,n_text_wds
item_id,,,,,,,,,,,,,,,,,,,,,
3b896605f03c,6.217606,1008,12,0,6.5,1.5,5.0,99914,914200,1433965,...,0.137931,0.000000,0.000000,0.103448,0.034483,0.000000,0.0,0.586207,48,13
c81c34d3aac5,4.616121,2909,5,1,15.0,2.0,2.0,58619,231290,1433965,...,0.125000,0.000000,0.000000,0.062500,0.062500,0.000000,0.0,1.187500,36,9
a8ab6225dd06,7.314887,527,32,1,NaN,NaN,NaN,85264,914200,1433965,...,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,22.000000,39,10
20bf5eea7fb5,5.861786,930,17,3,5.0,1.0,3.0,39932,72446,1433965,...,0.141593,0.053097,0.000000,0.150442,0.026549,0.017699,0.0,0.212389,32,25
e1cf6b9d5789,7.314887,657,7,1,15.0,2.0,6.0,43712,914200,1433965,...,0.163934,0.098361,0.065574,0.147541,0.049180,0.000000,0.0,0.393443,32,19


In [5]:
tr_c2v_matrix = load_matrix('tr_c2v_matrix')
tr_param_vec_matrix = load_matrix('tr_param_vec_matrix')
tr_txt_vec_matrix = load_matrix('tr_txt_vec_matrix')
tr_list = [daset_num_cols.loc[tr_index].values, tr_c2v_matrix, tr_param_vec_matrix, tr_txt_vec_matrix]
X = hstack(tr_list).tocsr()
dump_matrix(X, 'X_20180519')
del tr_list, tr_c2v_matrix,tr_param_vec_matrix,tr_txt_vec_matrix
gc.collect()

34

In [8]:
te_c2v_matrix = load_matrix('te_c2v_matrix')
te_param_vec_matrix = load_matrix('te_param_vec_matrix')
te_txt_vec_matrix = load_matrix('te_txt_vec_matrix')
te_list = [daset_num_cols.loc[te_index].values, te_c2v_matrix, te_param_vec_matrix, te_txt_vec_matrix]
X_te = hstack(te_list).tocsr()
dump_matrix(X_te, 'X_20180519_te')
del te_list, te_c2v_matrix, te_param_vec_matrix, te_txt_vec_matrix
gc.collect()

93

In [10]:
y = load_matrix('y')

In [11]:
feature_names = load_matrix('feature_names')
dump_matrix(feature_names, 'lgb_feature_20180519')
print('dataset shape:', X.shape, X_te.shape, len(y))
print('feature names:\n' + '\n'.join(feature_names[:20]))

dataset shape: (1503424, 60084) (508438, 60084) 1503424
feature names:
price
image_top_1
item_seq_number
n_missing_features
avg_days_up_user
avg_times_up_user
n_user_items
region_factor
parent_category_name_factor
user_type_factor
day_of_week_factor
n_region_target_avg
n_region_price_avg
n_image_top_1_target_avg
n_image_top_1_price_avg
n_category_name_target_avg
n_category_name_price_avg
n_user_type_target_avg
n_user_type_price_avg
n_parent_category_name_target_avg


In [12]:
del daset_num_cols
gc.collect()

21

In [13]:
del X, X_te, y, feature_names
gc.collect()

0